In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("./xlsx/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_Mid_irs','10Y_Mid_irs','1Y_Mid_crs','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [2]:
#전일 대비 변화량 계산
df["1Y_Mid_irs대비"] =df["1Y_Mid_irs"]-df["1Y_Mid_irs"].shift(1)
df["10Y_Mid_irs대비"] =df["10Y_Mid_irs"]-df["10Y_Mid_irs"].shift(1)
df["1Y_Mid_crs대비"] =df["1Y_Mid_crs"]-df["1Y_Mid_crs"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_Mid_irs,10Y_Mid_irs,1Y_Mid_crs,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_Mid_irs대비,10Y_Mid_irs대비,1Y_Mid_crs대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,,,,,
2012-08-01,1126.5,2.85,2.91,2.09,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,2.82,2.86,2.08,2.79,2.77,1134.00,2.5,-0.03,-0.05,-0.01,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,2.79,2.84,2.07,2.77,2.75,1138.00,2.4,-0.03,-0.02,-0.01,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,2.81,2.85,2.07,2.78,2.76,1128.50,2.4,0.02,0.01,0.00,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,2.82,2.87,2.09,2.77,2.75,1129.00,2.5,0.01,0.02,0.02,-0.01,-0.01,0.50,0.1


In [3]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns

Index(['종가_ex', '1Y_Mid_irs', '10Y_Mid_irs', '1Y_Mid_crs', '국고1년', '통안364일',
       'Mid_ndf', 'M1_스왑포인트', '1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비',
       '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비', '전날 종가_ex'],
      dtype='object')

In [4]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_Mid_irs대비', '10Y_Mid_irs대비','1Y_Mid_crs대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.158e+04
Date:                Fri, 19 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:23:12   Log-Likelihood:                -7660.3
No. Observations:                2459   AIC:                         1.534e+04
Df Residuals:                    2450   BIC:                         1.539e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.4520      2.462      1.808      0.071      -0.376       9.280
1Y_Mid_irs대비      6.3916      4.896      1.306      0.192      -3.209      15.992
10Y_Mid_irs대비    -2.8918      3.598     -0.804      0.422      -9.948       4.164
1Y_Mid_crs대비    -31.1868      2.262    -13.788      0.000     -35.622     -26.751
국고1년대비           22.1667      6.933      3.197      0.001       8.572      35.762
통안364일대비          0.1080      1.207      0.089      0.929      -2.260       2.476
Mid_ndf대비         0.0263      0.005      5.066      0.000       0.016       0.036
M1_스왑포인트         -0.1281      0.108     -1.188      0.235      -0.340       0.083
전날 종가_ex          0.9962      0.002    462.666      0.000       0.992       1.000
==============================================================================
Omnibus:                       44.133   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.644
Skew:                           0.009   Prob(JB):                     9.29e-20
Kurtosis:                       3.925   Cond. No.                     7.22e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# 종속변수 독립변수 의미없는값 삭제 재설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_Mid_crs대비', 'Mid_ndf대비', '전날 종가_ex']]

In [7]:
feature_add_std = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 8.381e+04
Date:                Fri, 19 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:23:15   Log-Likelihood:                -7668.7
No. Observations:                2459   AIC:                         1.535e+04
Df Residuals:                    2455   BIC:                         1.537e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.6560      2.262      1.174      0.240      -1.779       7.091
1Y_Mid_crs대비   -29.8894      2.239    -13.351      0.000     -34.279     -25.499
Mid_ndf대비        0.0266      0.005      5.112      0.000       0.016       0.037
전날 종가_ex         0.9977      0.002    501.232      0.000       0.994       1.002
==============================================================================
Omnibus:                       45.665   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.074
Skew:                           0.000   Prob(JB):                     1.01e-20
Kurtosis:                       3.948   Cond. No.                     2.37e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.37e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
X_train, X_test, y_train, y_test = train_test_split(feature_add_std,df_Target,test_size=0.2, random_state=16 )
lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse,mse

(5.103776717061415, 26.04853677761819)

In [9]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, df_Data, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.185346843880465,
 array([ 4.11553546,  6.70442593,  5.75608512, 19.1886816 ,  5.16200611]),
 array([ -16.93763212,  -44.949327  ,  -33.13251594, -368.2055015 ,
         -26.64630709]))